<a href="https://colab.research.google.com/github/chaiminwoo0223/Data-Analysis/blob/main/09%20-%20example_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install jpype1

In [ ]:
!pip install konlpy

In [ ]:
!pip install wordcloud

In [ ]:
import re
import konlpy
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import PIL

In [ ]:
# 데이터 불러오기
df = pd.read_csv('/content/drive/MyDrive/수업/데이터 분석 입문/강의자료/Data/news_comment_BTS.csv')
# 데이터 살펴보기
df.info()

In [ ]:
# 불필요한 문자 제거하기
df['reply'] = df['reply'].str.replace('[^가-힣]', ' ', regex = True)
df['reply'].head()

In [ ]:
# kkma 만들기
kkma = konlpy.tag.Kkma()

In [ ]:
# 명사 추출 - apply() 활용
nouns = df['reply'].apply(kkma.nouns)
nouns

In [ ]:
# 한 행에 한 단어가 들어가도록 구성
nouns = nouns.explode()
nouns

In [ ]:
# 데이터 프레임 만들기
df_word = pd.DataFrame({'word' : nouns})
# 글자 수 추가하기
df_word['count'] = df_word['word'].str.len()
# 두 글자 이상 단어만 남기기
df_word = df_word.query('count >= 2')
df_word

In [ ]:
# 빈도표 만들기
df_word = df_word.groupby('word', as_index = False)\
                 .agg(n = ('word', 'count'))\
                 .sort_values('n', ascending = False)
df_word

In [ ]:
# 단어 빈도 상위 20개 추출하기
top20 = df_word.head(20)
top20

In [ ]:
# 한글폰트 설정하기
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
plt.rcParams.update({'font.family'    : 'NanumBarunGothic',
                     'figure.dpi'     : '120',
                     'figure.figsize' : [6.5, 6]})
# 막대 그래프 만들기
sns.barplot(data = top20, y = 'word', x = 'n')

In [ ]:
dic_word = df_word.set_index('word').to_dict()['n']

In [ ]:
font = '/content/drive/MyDrive/수업/데이터 분석 입문/강의자료/Data/DoHyeon-Regular.ttf'

In [ ]:
icon = PIL.Image.open('/content/drive/MyDrive/수업/데이터 분석 입문/강의자료/Data/cloud.png')

In [ ]:
img = PIL.Image.new('RGB', icon.size, (255, 255, 255))
img.paste(icon, icon)
img = np.array(img)
img.shape

In [ ]:
# wc 만들기
from wordcloud import WordCloud
wc = WordCloud(random_state = 1234, font_path = font, width = 400, height = 400, background_color = 'white', mask = img)

In [ ]:
# 워드 클라우드 만들기
img_wordcloud = wc.generate_from_frequencies(dic_word)
# 워드 클라우드 출력하기
plt.figure(figsize = (10,10))
plt.axis('off')
plt.imshow(img_wordcloud)